In [14]:
# Can the network fit random labels

In [15]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

In [16]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

In [18]:
# Hyperparameters
input_size = 784  # MNIST image size (28x28) flattened
num_classes = 10  # Number of output classes for MNIST (digits 0-9)
num_epochs = 1000
batch_size = 64
learning_rate = 0.001

In [19]:
# Shuffle labels in the train dataset
np.random.seed(42)
# random_labels = np.random.permutation(len(train_dataset.targets))
random_labels = np.random.choice(10, len(train_dataset.targets)) # generate random labels between 0 and 9
train_dataset.targets = torch.Tensor(random_labels).long()

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [20]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = x.view(-1, input_size)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [21]:
# Initialize the model, loss function, and optimizer
model = DNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [22]:
# Training function
def train(model, optimizer, num_epochs):
    model.train()
    loss_list = []
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            # Forward pass
            output = model(data)
            loss = criterion(output, target)
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Add loss for each batch to total loss for the epoch
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        loss_list.append(avg_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

    return loss_list

In [23]:
# Test function
def test(model, optimizer):
    model.eval() 
    loss_list = []
    with torch.no_grad():
      for epoch in range(num_epochs):
        total_loss = 0
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
    
        avg_loss = total_loss / len(test_loader)
        loss_list.append(avg_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
    return loss_list

In [ ]:
# Train the model
train_losses = train(model, optimizer, num_epochs)
test_losses = test(model, optimizer)

Epoch [1/1000], Loss: 2.3037
Epoch [2/1000], Loss: 2.3029
Epoch [3/1000], Loss: 2.3023
Epoch [4/1000], Loss: 2.3016
Epoch [5/1000], Loss: 2.3001
Epoch [6/1000], Loss: 2.2975
Epoch [7/1000], Loss: 2.2936
Epoch [8/1000], Loss: 2.2873
Epoch [9/1000], Loss: 2.2795
Epoch [10/1000], Loss: 2.2691
Epoch [11/1000], Loss: 2.2561
Epoch [12/1000], Loss: 2.2406
Epoch [13/1000], Loss: 2.2234
Epoch [14/1000], Loss: 2.2034
Epoch [15/1000], Loss: 2.1827
Epoch [16/1000], Loss: 2.1618
Epoch [17/1000], Loss: 2.1391
Epoch [18/1000], Loss: 2.1177
Epoch [19/1000], Loss: 2.0934
Epoch [20/1000], Loss: 2.0694
Epoch [21/1000], Loss: 2.0462
Epoch [22/1000], Loss: 2.0229
Epoch [23/1000], Loss: 1.9997
Epoch [24/1000], Loss: 1.9769
Epoch [25/1000], Loss: 1.9538
Epoch [26/1000], Loss: 1.9342
Epoch [27/1000], Loss: 1.9128
Epoch [28/1000], Loss: 1.8903
Epoch [29/1000], Loss: 1.8700
Epoch [30/1000], Loss: 1.8502
Epoch [31/1000], Loss: 1.8327
Epoch [32/1000], Loss: 1.8118
Epoch [33/1000], Loss: 1.7963
Epoch [34/1000], Lo

In [ ]:
print(f"train_losses.shape: {len(train_losses)}")
print(f"test_losses.shape: {len(test_losses)}")

In [ ]:
# Plot the training and testing loss
plt.plot(range(1, num_epochs+1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs+1), test_losses, label='Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Testing Loss vs Epochs')
plt.show()